# Tutorial 07.0 - Keras Functional Model

Before you can start, you have to find a GPU on the system that is not heavily used by other users. Otherwise you cannot initialize your neural network.


**Hint:** the command is **nvidia-smi**, just in case it is displayed above in two lines because of a line break.

As a result you get a summary of the GPUs available in the system, their current memory usage (in MiB for megabytes), and their current utilization (in %). There should be six or eight GPUs listed and these are numbered 0 to n-1 (n being the number of GPUs). The GPU numbers (ids) are quite at the beginning of each GPU section and their numbers increase from top to bottom by 1.

Find a GPU where the memory usage is low. For this purpose look at the memory usage, which looks something like '365MiB / 16125MiB'. The first value is the already used up memory and the second value is the total memory of the GPU. Look for a GPU where there is a large difference between the first and the second value.

**Remember the GPU id and write it in the next line instead of the character X.**

In [1]:
# Change X to the GPU number you want to use,
# otherwise you will get a Python error
# e.g. USE_GPU = 4
USE_GPU = 4 # YOUR CHOICE

### Choose one GPU

**The following code is very important and must always be executed before using TensorFlow in the exercises, so that only one GPU is used and that it is set in a way that not all its memory is used at once. Otherwise, the other students will not be able to work with this GPU.**

The following program code imports the TensorFlow library for Deep Learning and outputs the version of the library.

Then, TensorFlow is configured to only see the one GPU whose number you wrote in the above cell (USE_GPU = X) instead of the X.

Finally, the GPU is set so that it does not immediately reserve all memory, but only uses more memory when needed. 

(The comments within the code cell explains a bit of what is happening if you are interested to better understand it. See also the documentation of TensorFlow for an explanation of the used methods.)

In [2]:
# Import TensorFlow 
import tensorflow as tf

# Print the installed TensorFlow version
print(f'TensorFlow version: {tf.__version__}\n')

# Get all GPU devices on this server
gpu_devices = tf.config.list_physical_devices('GPU')

# Print the name and the type of all GPU devices
print('Available GPU Devices:')
for gpu in gpu_devices:
    print(' ', gpu.name, gpu.device_type)
    
# Set only the GPU specified as USE_GPU to be visible
tf.config.set_visible_devices(gpu_devices[USE_GPU], 'GPU')

# Get all visible GPU  devices on this server
visible_devices = tf.config.get_visible_devices('GPU')

# Print the name and the type of all visible GPU devices
print('\nVisible GPU Devices:')
for gpu in visible_devices:
    print(' ', gpu.name, gpu.device_type)
    
# Set the visible device(s) to not allocate all available memory at once,
# but rather let the memory grow whenever needed
for gpu in visible_devices:
    tf.config.experimental.set_memory_growth(gpu, True)

TensorFlow version: 2.3.0

Available GPU Devices:
  /physical_device:GPU:0 GPU
  /physical_device:GPU:1 GPU
  /physical_device:GPU:2 GPU
  /physical_device:GPU:3 GPU
  /physical_device:GPU:4 GPU
  /physical_device:GPU:5 GPU
  /physical_device:GPU:6 GPU
  /physical_device:GPU:7 GPU

Visible GPU Devices:
  /physical_device:GPU:4 GPU


### Introduction to the functional API

So far all models have been implemented using the Sequential model. The Sequential model makes the assumption that the network has exactly one input and exactly one output, and that it consists of a linear stack of layers. However, this set of assumptions is too inflexible in a number of cases. Some networks require several independent inputs, some others require multiple outputs, and some networks have internal branching between layers making them look like graphs of layers rather than linear stacks of layers. 
These three important use cases—multi-input models, multi-output models, and graph-like models—are not possible when using only the Sequential model class in Keras. But there is also a different, far more general and flexible way to use Keras: the functional API. This sections explains in detail what it is, what it can do, and how to use it.

In the functional API, you are directly manipulating tensors, and you **use layers as functions that take tensors and return tensors** (hence the name "functional API").

In [3]:
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense

In [4]:
# this is a tensor 
input_tensor = Input(shape = (32,))

In [5]:
# A layer is a function.
dense = Dense(32, activation='relu')

In [6]:
# A layer may be called on a tensor, and it returns a tensor.
output_tensor = dense(input_tensor)

Let’s start with a minimal example: we will show side by side a simple Sequential model and its equivalent in the functional API.

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from keras import Input
# A Sequential model, which you already know all about.
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

In [8]:
seq_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Its functional equivalent.

In [9]:
from tensorflow.keras.models import Model 

In [10]:
input_tensor = Input(shape=(64,))  # defining the input tensor

x = layers.Dense(32, activation='relu')(input_tensor)  # defining a layer as function that takes as input a tesor
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

# is instantiating a Model object  using only an input tensor and output tensor
model = Model(input_tensor, output_tensor)

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


The most interesting part in the code above is **instantiating a Model object using only an input tensor and output tensor**. Behind the scenes, tensorflow will:
* retrieve every layer that was involved in going from input_tensor to output_tensor 
* bring them together into a graph-like data structure—a Model.
Of course, the reason it works is because **output_tensor was indeed obtained by repeatedly transforming input_tensor**. If you tried to build a model from inputs and outputs that were not related, you would
get a RuntimeError:

##### Training models built with the functional API: business as usual

In [ ]:
# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [ ]:
# Generate dummy Numpy data to train on
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))
# Train the model for 10 epochs
model.fit(x_train, y_train, epochs=10, batch_size=128)
# Evaluate the model
score = model.evaluate(x_train, y_train)

### Futher reading

https://www.tensorflow.org/guide/keras/functional